In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import torch
print("pytorch ver . ",torch.__version__)
print("Is CUDA available?",torch.cuda.is_available())
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import torch.utils.data as Data
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import xarray as xr
import os
os.environ['R_HOME'] = '/home/luhung3080/miniconda3/envs/chou/lib/R'
from rpy2.robjects import r, numpy2ri
numpy2ri.activate()
from rpy2.robjects.packages import importr
sinkr = importr('sinkr')
import Nio
print ("pynio ver . ",Nio.__version__)

/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pytorch ver .  1.11.0+cu113
Is CUDA available? True
pynio ver .  1.5.5


In [11]:
data=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/Data/newdata/result.csv')

In [12]:
###
#cal_PMf
###
u=np.zeros([605,5183])
for i in range (0,605):
    a=np.array(data['VALUE'][5183*i:5183*i+5183])
    for j in range(0,5183):
        if a[j]=='NaN' :
            a[j]=np.nan
    u[i]=a.T
print(np.shape(u))
###
#obs_PMf
###
v=np.zeros([605,5183])
for i in range (0,605):
    a=np.array(data['OBS_VALUE'][5183*i:5183*i+5183])
    for j in range(0,5183):
        if a[j]=='NaN' :
            a[j]=np.nan
        if a[j]=='A' :
            a[j]=np.nan
        if a[j]=='*' :
            a[j]=np.nan
        if a[j]=='x' :
            a[j]=np.nan  
    v[i]=a.T
    
print(np.shape(v)) 

(605, 5183)
(605, 5183)


In [13]:
XRestruct=sinkr.dineof(u)
YRestruct=sinkr.dineof(v)

[1] "1 EOF ; RMS = 7.7208659"
[1] "1 EOF ; RMS = 7.71915256"
[1] "1 EOF ; RMS = 7.71918485"
[1] "2 EOF ; RMS = 6.84092556"
[1] "2 EOF ; RMS = 6.84127094"
[1] "3 EOF ; RMS = 6.47602412"
[1] "3 EOF ; RMS = 6.4755135"
[1] "3 EOF ; RMS = 6.47549855"
[1] "3 EOF ; RMS = 6.47549755"
[1] "4 EOF ; RMS = 6.2521219"
[1] "4 EOF ; RMS = 6.25305477"
[1] "5 EOF ; RMS = 6.04839426"
[1] "5 EOF ; RMS = 6.0478386"
[1] "5 EOF ; RMS = 6.047815"
[1] "5 EOF ; RMS = 6.04781352"
[1] "6 EOF ; RMS = 5.86823128"
[1] "6 EOF ; RMS = 5.867931"
[1] "6 EOF ; RMS = 5.86791602"
[1] "6 EOF ; RMS = 5.86791411"
[1] "7 EOF ; RMS = 5.71967578"
[1] "7 EOF ; RMS = 5.71951606"
[1] "7 EOF ; RMS = 5.71952637"
[1] "8 EOF ; RMS = 5.599724"
[1] "8 EOF ; RMS = 5.60085644"
[1] "9 EOF ; RMS = 5.48613601"
[1] "9 EOF ; RMS = 5.48675208"
[1] "10 EOF ; RMS = 5.37241577"
[1] "10 EOF ; RMS = 5.37204237"
[1] "10 EOF ; RMS = 5.37202159"
[1] "10 EOF ; RMS = 5.37201994"
[1] "11 EOF ; RMS = 5.27154662"
[1] "11 EOF ; RMS = 5.27202257"
[1] "12 EOF 

In [14]:
XRestruct_Fun=np.array(XRestruct[0])
YRestruct_Fun=np.array(YRestruct[0])
print(np.shape(XRestruct_Fun))
print(np.shape(YRestruct_Fun))

(605, 5183)
(605, 5183)


In [15]:
Xhat=XRestruct_Fun
Yhat=YRestruct_Fun
Xhat_train = np.zeros([485,5183])
Yhat_train = np.zeros([485,5183])
Xhat_val = np.zeros([120,5183])
Yhat_val = np.zeros([120,5183])
Xhat_test = np.zeros([120,5183])
Yhat_test = np.zeros([120,5183])

for i in range (0,485):
    for j in range (0,5183):    
        Xhat_train[i][j] = Xhat[i][j]
        Yhat_train[i][j] = Yhat[i][j]
        
for i in range (485,605):
    for j in range (0,5183):
        Xhat_val[i-485][j] = Xhat[i][j]
        Yhat_val[i-485][j] = Yhat[i][j]     
        
for i in range (485,605):
    for j in range (0,5183):
        Xhat_test[i-485][j] = Xhat[i][j]
        Yhat_test[i-485][j] = Yhat[i][j]
print('Xhat_train')
print(np.shape(Xhat_train))
print('Yhat_train')
print(np.shape(Yhat_train))
print('Xhat_val')
print(np.shape(Xhat_val))
print('Yhat_val')
print(np.shape(Yhat_val))
print('Xhat_test')
print(np.shape(Xhat_test))
print('Yhat_test')
print(np.shape(Yhat_test))

Xhat_train
(485, 5183)
Yhat_train
(485, 5183)
Xhat_val
(120, 5183)
Yhat_val
(120, 5183)
Xhat_test
(120, 5183)
Yhat_test
(120, 5183)


In [8]:
class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

In [ ]:
# #Dlinear
# class DLinear_Model(nn.Module):
#     """
#     DLinear
#     """
#     def __init__(self, configs):
#         super(Model, self).__init__()
#         self.seq_len = configs.seq_len
#         self.pred_len = configs.pred_len

#         # Decompsition Kernel Size
#         kernel_size = 25
#         self.decompsition = series_decomp(kernel_size)
#         self.individual = configs.individual
#         self.channels = configs.enc_in

#         if self.individual:
#             self.Linear_Seasonal = nn.ModuleList()
#             self.Linear_Trend = nn.ModuleList()
#             self.Linear_Decoder = nn.ModuleList()
#             for i in range(self.channels):
#                 self.Linear_Seasonal.append(nn.Linear(self.seq_len,self.pred_len))
#                 self.Linear_Seasonal[i].weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
#                 self.Linear_Trend.append(nn.Linear(self.seq_len,self.pred_len))
#                 self.Linear_Trend[i].weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
#                 self.Linear_Decoder.append(nn.Linear(self.seq_len,self.pred_len))
#         else:
#             self.Linear_Seasonal = nn.Linear(self.seq_len,self.pred_len)
#             self.Linear_Trend = nn.Linear(self.seq_len,self.pred_len)
#             self.Linear_Decoder = nn.Linear(self.seq_len,self.pred_len)
#             self.Linear_Seasonal.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
#             self.Linear_Trend.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))

#     def forward(self, x):
#         # x: [Batch, Input length, Channel]
#         seasonal_init, trend_init = self.decompsition(x)
#         seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
#         if self.individual:
#             seasonal_output = torch.zeros([seasonal_init.size(0),seasonal_init.size(1),self.pred_len],dtype=seasonal_init.dtype).to(seasonal_init.device)
#             trend_output = torch.zeros([trend_init.size(0),trend_init.size(1),self.pred_len],dtype=trend_init.dtype).to(trend_init.device)
#             for i in range(self.channels):
#                 seasonal_output[:,i,:] = self.Linear_Seasonal[i](seasonal_init[:,i,:])
#                 trend_output[:,i,:] = self.Linear_Trend[i](trend_init[:,i,:])
#         else:
#             seasonal_output = self.Linear_Seasonal(seasonal_init)
#             trend_output = self.Linear_Trend(trend_init)

#         x = seasonal_output + trend_output
#         return x.permute(0,2,1) # to [Batch, Output length, Channel]

In [ ]:
# x_train = torch.from_numpy(Xhat_train).float()
# y_train = torch.from_numpy(Yhat_train).float()
# x_val = torch.from_numpy(Xhat_val).float()
# y_val = torch.from_numpy(Yhat_val).float()
# x_test = torch.from_numpy(Xhat_test).float()
# y_test = torch.from_numpy(Yhat_test).float()

In [9]:
def model(self, x):
    # x: [Batch, Input length, Channel]
    seasonal_init, trend_init = self.decompsition(x)
    seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
    if self.individual:
        seasonal_output = torch.zeros([seasonal_init.size(0),seasonal_init.size(1),self.pred_len],dtype=seasonal_init.dtype).to(seasonal_init.device)
        trend_output = torch.zeros([trend_init.size(0),trend_init.size(1),self.pred_len],dtype=trend_init.dtype).to(trend_init.device)
        for i in range(self.channels):
            seasonal_output[:,i,:] = self.Linear_Seasonal[i](seasonal_init[:,i,:])
            trend_output[:,i,:] = self.Linear_Trend[i](trend_init[:,i,:])
    else:
        seasonal_output = self.Linear_Seasonal(seasonal_init)
        trend_output = self.Linear_Trend(trend_init)

    x = seasonal_output + trend_output
    return x.permute(0,2,1) # to [Batch, Output length, Channel]

In [20]:
features = torch.from_numpy(Xhat_train)
targets = torch.from_numpy(Yhat_train)
x_test = torch.from_numpy(Xhat_test)
y_test = torch.from_numpy(Yhat_test)


rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epo = 2001
loss = nn.L1Loss()
train_error = np.zeros(epo)
test_error = np.zeros(epo)


for epoch in range (epo):
    yhats_train = model(features.float())
    train_loss = loss(targets.float() , yhats_train)
    train_error[epoch] = train_loss

    train_loss.backward() 
    optimizer.step()      
    optimizer.zero_grad()
  

    if epoch <= 10 or epoch % 10 == 0:
        print(f"Epoch {epoch}, Training loss {train_loss.item():.4f}")
    else :
        if epoch >= epo-10 :
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f}")

AttributeError: 'function' object has no attribute 'parameters'

In [18]:
class LinearRegression(nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        
        # define layers
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
    
        return self.linear(x)

In [19]:
# 0) prepare data
feature_numpy, target_numpy = datasets.make_regression(n_samples=100, n_features=1, noise=20, random_state=1234)

feature = torch.from_numpy(feature_numpy.astype(np.float32))
target = torch.from_numpy(target_numpy.astype(np.float32))
target = target.view(target.shape[0], 1)

n_samples, n_features = feature.shape

# 1) model
class LinearRegression(nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        
        # define layers
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
    
        return self.linear(x)

model = LinearRegression(n_features, 1)

# 2) loss and optimizer
learning_rate = 0.01
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) training loop
epochs = 100
for epoch in range(epochs):
    # forward pass and loss
    y_predicted = model(feature)
    loss = criterion(y_predicted, target)

    # backward pass
    loss.backward()

    # update
    optimizer.step()
    
    # init optimizer
    optimizer.zero_grad()

    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item(): .4f}')

# show in image
predicted = model(feature).detach().numpy()
plt.plot(feature_numpy, target_numpy, 'ro')
plt.plot(feature_numpy, predicted, 'b')
plt.show()

NameError: name 'datasets' is not defined